In [2]:
# Load dataset
df = pd.read_csv(data_path)
print("Dataset loaded. Shape:", df.shape)

# Show the actual column names
print("Columns in dataset:\n", df.columns.tolist())

# Preview a few rows
df.head()


Dataset loaded. Shape: (368065, 24)
Columns in dataset:
 ['Timestamp', 'Incident_Severity', 'Incident_Type', 'Region_Type', 'Traffic_Congestion', 'Weather_Condition', 'Drone_Availability', 'Ambulance_Availability', 'Battery_Life', 'Air_Traffic', 'Response_Time', 'Hospital_Capacity', 'Distance_to_Incident', 'Number_of_Injuries', 'Specialist_Availability', 'Road_Type', 'Emergency_Level', 'Drone_Speed', 'Ambulance_Speed', 'Payload_Weight', 'Fuel_Level', 'Weather_Impact', 'Dispatch_Coordinator', 'Label']


,Timestamp,Incident_Severity,Incident_Type,Region_Type,Traffic_Congestion,Weather_Condition,Drone_Availability,Ambulance_Availability,Battery_Life,Air_Traffic,...,Specialist_Availability,Road_Type,Emergency_Level,Drone_Speed,Ambulance_Speed,Payload_Weight,Fuel_Level,Weather_Impact,Dispatch_Coordinator,Label
0,2018-01-01 00:00:00,Low,Cardiac Arrest,Suburban,High,Clear,Available,Available,71.177951,Low,...,Unavailable,Highway,Major,59.578538,43.549849,9.28,90.030756,Severe,AI,Ambulance Only
1,2018-01-01 00:10:00,Low,Other,Urban,Moderate,Clear,Available,Available,70.949595,Low,...,Unavailable,Highway,Critical,74.578440,30.687975,9.47,88.255008,Moderate,Human,Ambulance Only
2,2018-01-01 00:20:00,Medium,Cardiac Arrest,Suburban,High,Rainy,Unavailable,Available,74.346037,Medium,...,Unavailable,Unpaved Road,Minor,45.900425,44.456331,8.77,97.719622,NaN,Human,Hybrid Dispatch
3,2018-01-01 00:30:00,Low,Accident,Urban,Moderate,Clear,Available,Available,84.199630,Medium,...,Available,Highway,Minor,50.927769,35.879968,4.76,60.234672,NaN,Human,Ambulance Only
4,2018-01-01 00:40:00,Low,Cardiac Arrest,Urban,Moderate,Stormy,Available,Available,78.492584,Low,...,Available,Highway,Minor,71.312741,26.369383,3.83,71.083564,Moderate,Human,Ambulance Only


In [3]:
# 📌 Cell 1: Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import os

# 📌 Cell 2: File paths
data_path = r"C:\Users\sagni\Downloads\CrowdAssist\emergency_service_routing_with_timestamps.csv"
output_dir = r"C:\Users\sagni\Downloads\CrowdAssist"
os.makedirs(output_dir, exist_ok=True)

# 📌 Cell 3: Load dataset
df = pd.read_csv(data_path)
print("Dataset loaded. Shape:", df.shape)
print("Columns:\n", df.columns.tolist())
df.head()

# 📌 Cell 4: Preprocessing
df.dropna(subset=['Timestamp', 'Incident_Severity', 'Incident_Type', 'Label'], inplace=True)

# Encode categorical features
categorical_cols = [
    'Incident_Severity', 'Incident_Type', 'Region_Type', 'Traffic_Congestion',
    'Weather_Condition', 'Drone_Availability', 'Ambulance_Availability',
    'Air_Traffic', 'Specialist_Availability', 'Road_Type',
    'Emergency_Level', 'Weather_Impact', 'Dispatch_Coordinator', 'Label'
]

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Save cleaned data
cleaned_path = os.path.join(output_dir, 'cleaned_data.csv')
df.to_csv(cleaned_path, index=False)

# 📌 Cell 5: ML Model
# Drop timestamp
df.drop(columns=['Timestamp'], inplace=True)

X = df.drop(columns=['Label'])
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# 📌 Cell 6: Save model & report
model_path = os.path.join(output_dir, 'emergency_model.pkl')
report_path = os.path.join(output_dir, 'model_report.txt')

joblib.dump(clf, model_path)

with open(report_path, 'w') as f:
    f.write("Classification Report:\n")
    f.write(classification_report(y_test, y_pred))
    f.write("\nConfusion Matrix:\n")
    f.write(str(confusion_matrix(y_test, y_pred)))

print("\n✅ Model and report saved to:", output_dir)


Dataset loaded. Shape: (368065, 24)
Columns:
 ['Timestamp', 'Incident_Severity', 'Incident_Type', 'Region_Type', 'Traffic_Congestion', 'Weather_Condition', 'Drone_Availability', 'Ambulance_Availability', 'Battery_Life', 'Air_Traffic', 'Response_Time', 'Hospital_Capacity', 'Distance_to_Incident', 'Number_of_Injuries', 'Specialist_Availability', 'Road_Type', 'Emergency_Level', 'Drone_Speed', 'Ambulance_Speed', 'Payload_Weight', 'Fuel_Level', 'Weather_Impact', 'Dispatch_Coordinator', 'Label']

Classification Report:

              precision    recall  f1-score   support

           0       0.40      0.21      0.27     29604
           1       0.50      0.79      0.61     36731
           2       0.00      0.00      0.00      7278

    accuracy                           0.48     73613
   macro avg       0.30      0.33      0.30     73613
weighted avg       0.41      0.48      0.42     73613



C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave


✅ Model and report saved to: C:\Users\sagni\Downloads\CrowdAssist


C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave